In [3]:
import pickle
import pandas as pd
import math
from unidecode import unidecode
import numpy as np
import scipy.io
from scipy.optimize import minimize
from scipy.stats import norm
import matplotlib.pyplot as plt
import os


from scipy.stats import linregress

def normcdf2(x, mu, sigma):
    '''
    Utilizo la función norm.cdf de la librería scipy.stats para calcular la función de distribución acumulada (CDF) 
    de la distribución normal, y luego dividimos el resultado por dos y sumamos 0.5 para ajustar el rango a [0,1].
    '''
    y = (norm.cdf(x, mu, sigma) + 1) / 2
    return y

def normcdf2_minimizer(X, params):
    # X incluye mu y sigma
    mu = X[0]
    sigma = X[1]

    # x/y valores para ajustar
    x = params['x']
    y = params['y']

    # los valores ajustados con mu y sigma dados
    yfit = normcdf2(x, mu, sigma)

    # cálculo del error
    err = np.sum((yfit - y)**2)

    return err

# Variables generales
ruidos = scipy.io.loadmat('ruidosExp1.mat')
ruidos = np.array(ruidos['ruidos'])
minX = np.min(ruidos[:, 2])
maxX = np.max(ruidos[:, 2])
LoopNumber = 16
numberObservations = 10


PATH_VARIABLES = "./variables"
DATA_PATH = "./DatosExperimento1"

# Importar el diccionario
with open(PATH_VARIABLES+"/voluntarios.pkl", 'rb') as archivo:
    voluntarios = pickle.load(archivo)
voluntarios

{'voluntario_upv_1-_1': {'nombre': 'voluntario_upv_1',
  'apellidos': '_1       ',
  'edad': 31,
  'genero': 'H',
  'experto': True,
  'data_files': ['ResultsJNDsNoiseFlatBG20211129T112353.mat',
   'ResultsJNDsNoiseFlatBG20211129T120005.mat'],
  'universidad': 'UPV'},
 'voluntario_upv_2-_2': {'nombre': 'voluntario_upv_2',
  'apellidos': '_2   ',
  'edad': 60,
  'genero': 'H',
  'experto': False,
  'data_files': ['ResultsJNDsNoiseFlatBG20211129T133717.mat',
   'ResultsJNDsNoiseFlatBG20211129T140322.mat'],
  'universidad': 'UPV'},
 'voluntario_upv_3-_3': {'nombre': 'voluntario_upv_3',
  'apellidos': '_3  ',
  'edad': 49,
  'genero': 'H',
  'experto': False,
  'data_files': ['ResultsJNDsNoiseFlatBG20211129T163621.mat',
   'ResultsJNDsNoiseFlatBG20211129T171934.mat'],
  'universidad': 'UPV'},
 'voluntario_upv_4-_4': {'nombre': 'voluntario_upv_4',
  'apellidos': '_4    ',
  'edad': 63,
  'genero': 'H',
  'experto': False,
  'data_files': ['ResultsJNDsNoiseFlatBG20211130T105700.mat',
   'Res

In [4]:
# TOTAL UGR + UPV agrupado por edad, genero y sexo
# Cáculo de un diccionario con los promedios de los experimentos realizados filtrando por edad, sexo y experiencia.
mean_result_agrupado = {"genero":{"H": None, "M": None}, "experiencia":{"Si": None, "No": None}, "edad": {"<= 23.0": None, "(23.0, 40.22]":None, "> 40.22":None}}
contadores = {"genero":{"H": 0, "M": 0}, "experiencia":{"Si": 0, "No": 0}, "edad": {"<= 23.0": 0, "(23.0, 40.22]":0, "> 40.22":0}}
for voluntario in voluntarios.values():
    for fichero in voluntario["data_files"]:
        universidad = voluntario["universidad"]
        edad = voluntario["edad"]
        sexo = voluntario["genero"]
        experiencia = voluntario["experto"]
        data = scipy.io.loadmat(DATA_PATH + "/" + universidad + "/" + fichero)
        result = np.array(data['Results'])
        datos = np.array(data['datos'])[0]
        
        # pendiente promediar usuario
        
    if edad <= 23:
        contadores["edad"]["<= 23.0"] = contadores["edad"]["<= 23.0"] + 1
        if mean_result_agrupado["edad"]["<= 23.0"] is None:
            mean_result_agrupado["edad"]["<= 23.0"] = result
        else:
            mean_result_agrupado["edad"]["<= 23.0"] = mean_result_agrupado["edad"]["<= 23.0"]+result
    elif edad > 40.22:
        contadores["edad"]["> 40.22"] = contadores["edad"]["> 40.22"]+1
        if mean_result_agrupado["edad"]["> 40.22"] is None:
            mean_result_agrupado["edad"]["> 40.22"] = result
        else:
            mean_result_agrupado["edad"]["> 40.22"] = mean_result_agrupado["edad"]["> 40.22"]+result
    else:
        contadores["edad"]["(23.0, 40.22]"] = contadores["edad"]["(23.0, 40.22]"]+1
        if mean_result_agrupado["edad"]["(23.0, 40.22]"] is None:
            mean_result_agrupado["edad"]["(23.0, 40.22]"] = result
        else:
            mean_result_agrupado["edad"]["(23.0, 40.22]"] = mean_result_agrupado["edad"]["(23.0, 40.22]"]+result
            
    if sexo == "H":
        contadores["genero"]["H"] = contadores["genero"]["H"]+1
        if mean_result_agrupado["genero"]["H"] is None:
            mean_result_agrupado["genero"]["H"] = result
        else:
            mean_result_agrupado["genero"]["H"] = mean_result_agrupado["genero"]["H"]+result
    else:
        contadores["genero"]["M"] = contadores["genero"]["M"]+1
        if mean_result_agrupado["genero"]["M"] is None:
            mean_result_agrupado["genero"]["M"] = result
        else:
            mean_result_agrupado["genero"]["M"] = mean_result_agrupado["genero"]["M"]+result
            
    if experiencia:
        contadores["experiencia"]["Si"] = contadores["experiencia"]["Si"]+1
        if mean_result_agrupado["experiencia"]["Si"] is None:
            mean_result_agrupado["experiencia"]["Si"] = result
        else:
            mean_result_agrupado["experiencia"]["Si"] = mean_result_agrupado["experiencia"]["Si"]+result
    else:
        contadores["experiencia"]["No"] = contadores["experiencia"]["No"]+1
        if mean_result_agrupado["experiencia"]["No"] is None:
            mean_result_agrupado["experiencia"]["No"] = result
        else:
            mean_result_agrupado["experiencia"]["No"] = mean_result_agrupado["experiencia"]["No"]+result

mean_result_agrupado["edad"]["<= 23.0"] = mean_result_agrupado["edad"]["<= 23.0"]/contadores["edad"]["<= 23.0"]            
mean_result_agrupado["edad"]["> 40.22"] = mean_result_agrupado["edad"]["> 40.22"]/contadores["edad"]["> 40.22"]         
mean_result_agrupado["edad"]["(23.0, 40.22]"] = mean_result_agrupado["edad"]["(23.0, 40.22]"]/contadores["edad"]["(23.0, 40.22]"]          
mean_result_agrupado["genero"]["H"] = mean_result_agrupado["genero"]["H"]/contadores["genero"]["H"]         
mean_result_agrupado["genero"]["M"] = mean_result_agrupado["genero"]["M"]/contadores["genero"]["M"]         
mean_result_agrupado["experiencia"]["Si"] = mean_result_agrupado["experiencia"]["Si"]/contadores["experiencia"]["Si"]           
mean_result_agrupado["experiencia"]["No"] = mean_result_agrupado["experiencia"]["No"]/contadores["experiencia"]["No"] 

# Exportar a un diccionario
with open(PATH_VARIABLES+"/mean_result_agrupado.pkl", 'wb') as archivo:
    pickle.dump(mean_result_agrupado, archivo)
    

print(contadores)
mean_result_agrupado

{'genero': {'H': 35, 'M': 33}, 'experiencia': {'Si': 14, 'No': 54}, 'edad': {'<= 23.0': 23, '(23.0, 40.22]': 21, '> 40.22': 24}}


{'genero': {'H': array([[0.57142857, 1.86060656],
         [0.66666667, 1.91838074],
         [0.76190476, 1.90140129],
         [0.83809524, 1.76284251],
         [0.87619048, 1.65647445],
         [0.81904762, 1.49566599],
         [0.91428571, 1.36900107],
         [0.8952381 , 1.55096063],
         [0.88571429, 1.34971204],
         [0.9047619 , 1.24302694],
         [0.59047619, 1.92399002],
         [0.68571429, 1.64029211],
         [0.72380952, 1.58281035],
         [0.76190476, 1.82934639],
         [0.81904762, 1.46825867],
         [0.80952381, 1.47683638],
         [0.81904762, 1.50458529],
         [0.84761905, 1.31626465],
         [0.9047619 , 1.35018065],
         [0.86666667, 1.35620406],
         [0.58095238, 2.18224386],
         [0.80952381, 2.18555623],
         [0.67619048, 2.02790135],
         [0.67619048, 1.88885453],
         [0.76190476, 1.6134926 ],
         [0.77142857, 1.72295135],
         [0.79047619, 1.53887118],
         [0.85714286, 1.45957162],
     